In [ ]:
!pip -q install langchain langchain-openai langchain-community chromadb gradio

In [ ]:
import os

# OpenAI API 키 설정 (사용자가 입력해야 함)
# from google.colab import userdata
# api_key=userdata.get('api_key')
# os.environ["OPENAI_API_KEY"] = api_key
# api_key2=userdata.get('api_key2')
# os.environ["LANGCHAIN_API_KEY"] = api_key2

from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
OPENAPI_KEY = os.getenv("OPENAI_API_KEY")
LangSmith_KEY = os.getenv("LANGCHAIN_API_KEY")

# 2) LangSmith 연동 필수 환경변수
os.environ["LANGCHAIN_TRACING_V2"] = "true"      # 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"]   = "https://api.smith.langchain.com"  # 기본값
os.environ["LANGSMITH_PROJECT"]    = "AGENT_ret_1"                 # 수업용 프로젝트명

In [5]:
# ============================================
# 3. 모듈 임포트
# ============================================
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.tools import Tool
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage # Import necessary message types

# ============================================
# 4. 뉴스 기사 데이터 (샘플)
# ============================================
news_articles = [
    Document(page_content="삼성전자가 반도체 시장에서 매출 1위를 기록했다."),
    Document(page_content="현대자동차는 전기차 신모델을 출시하며 글로벌 시장을 공략하고 있다."),
    Document(page_content="카카오가 새로운 AI 챗봇 서비스를 공개했다."),
]

# ============================================
# 5. 임베딩 + 벡터스토어
# ============================================
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_documents(news_articles, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# ============================================
# 6. Retriever Tool 정의 (뉴스 요약)
# ============================================
def search_and_summarize(query: str) -> str:
    docs = retriever.get_relevant_documents(query)
    context = "\n".join([d.page_content for d in docs])
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    prompt = f"다음 뉴스 기사를 참고하여 '{query}'에 대해 3문장으로 요약해줘:\n{context}"
    return llm.invoke(prompt).content

news_tool = Tool(
    name="news_search",
    description="한국 뉴스 기사를 검색하고 요약한다.",
    func=search_and_summarize
)

# ============================================
# 7. Agent 설정
# ============================================
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
tools = [news_tool]

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 한국 경제 뉴스를 분석하는 전문가이다."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# ============================================
# 8. 실행 예시
# ============================================
print(agent_executor.invoke({"input": "삼성전자 소식 알려줘", "chat_history": []})["output"])



> Entering new AgentExecutor chain...

Invoking: `news_search` with `삼성전자`




/tmp/ipython-input-4105117717.py:32: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


삼성전자가 반도체 시장에서 매출 1위를 기록했다. 이는 회사의 기술력과 시장 점유율을 반영하는 성과로, 글로벌 경쟁에서의 우위를 나타낸다. 앞으로도 삼성전자는 지속적인 혁신과 투자로 반도체 분야에서의 리더십을 강화할 계획이다.최근 삼성전자는 반도체 시장에서 매출 1위를 기록했습니다. 이는 회사의 기술력과 시장 점유율을 반영하는 성과로, 글로벌 경쟁에서의 우위를 나타냅니다. 앞으로도 삼성전자는 지속적인 혁신과 투자로 반도체 분야에서의 리더십을 강화할 계획이라고 합니다.

> Finished chain.
최근 삼성전자는 반도체 시장에서 매출 1위를 기록했습니다. 이는 회사의 기술력과 시장 점유율을 반영하는 성과로, 글로벌 경쟁에서의 우위를 나타냅니다. 앞으로도 삼성전자는 지속적인 혁신과 투자로 반도체 분야에서의 리더십을 강화할 계획이라고 합니다.
